### Business Problem

I just finished my bachelors degree and I have 4 job offers in Manila, I’m currently living in Abra which is around 8 hours drive to Manila. Provincial Life is very different from City life therefore before I choose which job should I pick I want to be able to determine which workplace is accessible to different amenities I require in order to survive. 

This project can be useful to anyone who has never been to a place and would like a place with accessible amenities.

### Data Approach

I only have the location of the 4 workplaces, in order for me to decide which workplace is the best I will be using 4 square to get the nearby places where I can stay and will run foursquare again on this establishments to see the accessible/nearby amenities.

### workplace with location

In [10]:
work_loc = {'address':['Cybergate Tower 1', 'Tech Mahindra', 'Glass Tower', 'SM Mall of Asia'],
           'lat': ['14.57082', '14.607486', '14.555066', '14.535043'],
           'lon': ['121.048411', '121.079501', '121.018639', "120.982712"]}

### Convert Dictionary to DataFrame

In [15]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [16]:
df = pd.DataFrame.from_dict(work_loc)
df

,address,lat,lon
0,Cybergate Tower 1,14.57082,121.048411
1,Tech Mahindra,14.607486,121.079501
2,Glass Tower,14.555066,121.018639
3,SM Mall of Asia,14.535043,120.982712


### Import map plotting library

In [18]:
import folium

### Plot Job Location In Map

In [33]:
## set map center
center = [' 14.566998', '121.028143']

map_center = folium.Map(location=[center[0], center[1]], zoom_start=12)

# add markers to map
for lat, lng, address in zip(df['lat'], df['lon'], df['address']):
    label = '{}'.format(address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#FF0000',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_center)  
    
map_center

### Methodology

In order for me to decide which of the workplace is the best [based on location], I will be using foursquare data to get what are the nearby amenities / places around KM, next step is I will be filtering the result if there are nearby residential building or condominium, if these criteria are not met we will stop the exploration, If there are nearby residential area or condo, next step is we will get location of this building/s and run the foursquare API again to get the nearby places in this we will limit the radius to 200M. after this we will process the data to get the variety of buildings that are nearby, after this we can now come into conclusion

### foursquare credentials and version

In [34]:
CLIENT_ID = 'WO5VHHRQ1YQCISVZ1RHUPMPMKQV1EL3HLUCFJE02N4YIX01E'
CLIENT_SECRET = 'OZGOUJU3PA2BRQA4HNNXCAWBJTCP33WS2SXXER552SXVWF5A' 
VERSION = '20191001' # Foursquare API version

In [36]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
def get_nearby(lat, lon, radius, limit):

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lon, 
        radius, 
        limit)
    
    results = requests.get(url).json()
    
    venues = results['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    
    return nearby_venues

### Now that I've prepared all necesarry things to find what are the nearby places / amenities is my four possible workplace we now run function with 4 square API

In [44]:
nearby_workplace_1 = get_nearby(df.iloc[0,1], df.iloc[0,2], radius = 1000, limit = 100)
nearby_workplace_1

,name,categories,lat,lng
0,Subway,Sandwich Place,14.571318,121.049638
1,Nuat Thai,Massage Studio,14.571765,121.050081
2,Starbucks,Coffee Shop,14.570962,121.048611
3,ACTS Dance and Arts Academy,Dance Studio,14.573937,121.047707
4,The Legend Villas,Hotel,14.571211,121.050824
5,Dunkin' Donuts,Donut Shop,14.572262,121.046951
6,Rolling Rice,Japanese Restaurant,14.571845,121.049805
7,SPARTA Philippines,Soccer Field,14.570875,121.053120
8,Azul Spa,Spa,14.573960,121.048266
9,Sonsi House of Longganisa,Filipino Restaurant,14.570439,121.052258


In [47]:
cybergate = folium.Map(location=[df.iloc[0,1], df.iloc[0,2]], zoom_start=20)

# add markers to map
for lat, lng, name in zip(nearby_workplace_1['lat'], nearby_workplace_1['lng'], nearby_workplace_1['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='##ffff00',
        fill=True,
        fill_color='#ffff00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_center)  
    
map_center

### try to look for residencial building that is in nearby 

In [49]:
residence = nearby_workplace_1[nearby_workplace_1['categories'].str.contains("residence")]
residence

,name,categories,lat,lng


In [50]:
condo = nearby_workplace_1[nearby_workplace_1['categories'].str.contains("condo")]
condo

,name,categories,lat,lng


### Since there are no nearby condo or residencial area near cybergate 1 [based on four square nearby] we wil skip the next step where we will get what is nearby in the residences area

## NEXT
   ### we will now proceed to possible workplace 2

In [51]:
nearby_workplace_2 = get_nearby(df.iloc[1,1], df.iloc[1,2], radius = 1000, limit = 200)
nearby_workplace_2

,name,categories,lat,lng
0,House of Wagyu Stone Grill,Australian Restaurant,14.608528,121.078888
1,Healthy Options,Health Food Store,14.608519,121.078886
2,Built By Sonic,Clothing Store,14.609758,121.081120
3,Mystery Manila,Recreation Center,14.606629,121.078877
4,Eastwood City,Business Center,14.609450,121.079849
5,Nail Tropics Eastwood,Cosmetics Shop,14.610034,121.078804
6,Señor Pollo,Latin American Restaurant,14.609550,121.080812
7,TGI Fridays,American Restaurant,14.609317,121.080488
8,Eastwood Citywalk 1,Shopping Mall,14.608452,121.078900
9,Gloss Nail & Body Spa,Spa,14.608651,121.081019


In [52]:
tech_mahindra = folium.Map(location=[df.iloc[1,1], df.iloc[1,2]], zoom_start=20)

# add markers to map
for lat, lng, name in zip(nearby_workplace_2['lat'], nearby_workplace_2['lng'], nearby_workplace_2['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#228B22',
        fill=True,
        fill_color='#228B22',
        fill_opacity=0.7,
        parse_html=False).add_to(map_center)  
    
map_center

### follow the same step as the previous

In [54]:
residence = nearby_workplace_2[nearby_workplace_2['categories'].str.contains("residence")]
residence

,name,categories,lat,lng


In [57]:
condo = nearby_workplace_2[nearby_workplace_2['categories'].str.contains("Condo")]
condo

,name,categories,lat,lng
60,One Central Park,Residential Building (Apartment / Condo),14.610219,121.079222


### now that we have residential area result we can now proceed to next step where we will get what are the nearby amenities / places around 200 m in this area

In [68]:
one_central = get_nearby(condo.iloc[0,2], condo.iloc[0,3], radius = 200, limit = 100)
one_central.head()

,name,categories,lat,lng
0,House of Wagyu Stone Grill,Australian Restaurant,14.608528,121.078888
1,Healthy Options,Health Food Store,14.608519,121.078886
2,Nail Tropics Eastwood,Cosmetics Shop,14.610034,121.078804
3,Café Mary Grace,Café,14.610511,121.080081
4,Señor Pollo,Latin American Restaurant,14.609550,121.080812


In [87]:
# group by categories
r = one_central.groupby('categories').count().sort_values(by = ['name'], ascending = False).reset_index()
r.head(15)

,categories,name,lat,lng
0,Japanese Restaurant,3,3,3
1,Movie Theater,3,3,3
2,Filipino Restaurant,3,3,3
3,Spa,3,3,3
4,Clothing Store,3,3,3
5,Coffee Shop,2,2,2
6,Portuguese Restaurant,2,2,2
7,Italian Restaurant,2,2,2
8,Cosmetics Shop,2,2,2
9,Ramen Restaurant,2,2,2


#### base on this result we can say that this residence place is good because there are 2 shopping mall and restaurants which offers different cuisine

### we will now proceed on the third location

In [89]:
nearby_workplace_3 = get_nearby(df.iloc[2,1], df.iloc[2,2], radius = 1000, limit = 200)
nearby_workplace_3

,name,categories,lat,lng
0,Wildflour Café + Bakery,Café,14.555582,121.018130
1,Bliss Yoga Manila,Yoga Studio,14.557161,121.018243
2,"The Curator, Coffee & Cocktails",Café,14.554463,121.017994
3,"Touch And Heal, Rufino St, Makati",Massage Studio,14.555641,121.017668
4,The Coffee Bean & Tea Leaf,Coffee Shop,14.555795,121.018240
5,The Belle & Dragon,Gastropub,14.555830,121.019435
6,Bistro Pamana,Filipino Restaurant,14.554566,121.020456
7,Makati Diamond Residences,Residential Building (Apartment / Condo),14.553794,121.020856
8,Toby's Estate Coffee Roasters,Coffee Shop,14.554619,121.016679
9,Purple Oven,Bakery,14.554673,121.016774


In [90]:
glass_tower = folium.Map(location=[df.iloc[2,1], df.iloc[2,2]], zoom_start=20)

# add markers to map
for lat, lng, name in zip(nearby_workplace_3['lat'], nearby_workplace_3['lng'], nearby_workplace_3['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#0000ff ',
        fill=True,
        fill_color='#0000ff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_center)  
    
map_center

In [92]:
residence = nearby_workplace_3[nearby_workplace_3['categories'].str.contains("residence")]
residence

,name,categories,lat,lng


In [94]:
condo = nearby_workplace_3[nearby_workplace_3['categories'].str.contains("condo")]
condo

,name,categories,lat,lng


### Since we are also not getting any residential place we well skip the proceeding steps

In [95]:
nearby_workplace_4 = get_nearby(df.iloc[3,1], df.iloc[3,2], radius = 1000, limit = 200)
nearby_workplace_4

,name,categories,lat,lng
0,Yabu: House of Katsu,Japanese Restaurant,14.536158,120.981613
1,Forever 21,Clothing Store,14.535021,120.982607
2,Yellow Cab Pizza Co.,Pizza Place,14.536263,120.982020
3,Uniqlo ユニクロ,Clothing Store,14.535523,120.982589
4,Chili's,Mexican Restaurant,14.534165,120.982466
5,Cafe Mary Grace,Café,14.534304,120.981713
6,Yayoi,Japanese Restaurant,14.534206,120.981682
7,Cold Stone Creamery,Ice Cream Shop,14.535398,120.982601
8,Jamba Juice,Juice Bar,14.535307,120.981759
9,Conrad Manila,Hotel,14.532655,120.980661


In [96]:
mall_of_asia = folium.Map(location=[df.iloc[3,1], df.iloc[3,2]], zoom_start=20)

# add markers to map
for lat, lng, name in zip(nearby_workplace_4['lat'], nearby_workplace_4['lng'], nearby_workplace_4['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#8000ff ',
        fill=True,
        fill_color='#8000ff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_center)  
    
map_center

In [97]:
residence = nearby_workplace_4[nearby_workplace_4['categories'].str.contains("residence")]
residence

,name,categories,lat,lng


In [98]:
condo = nearby_workplace_4[nearby_workplace_4['categories'].str.contains("condo")]
condo

,name,categories,lat,lng


### since we are also not getting any residential area nearby we will now end our exploration

## Conclusion
   ### since we run the the on all the four possible workplace location and it seems like only one of them have residential building arounf 1KM therefore we can say that tech mahindra have the nices location to work considering how heavy the traffic in Manila, working in a place where everything is accessible something that everyone could wish for.

# Thank YOU!